In [ ]:
##############################
# probs0.json
##############################

import requests
import time
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import random

tstart = time.time()
print(tstart)

api_url = 'https://api.hooktheory.com/v1/'
login = {"username": "jl8429",
         "password": ""}

r = requests.post(api_url+'users/auth', data=login)
#print('r', r)
#print(r.json())
'''
{'id': 136206, 'username': 'myUsername', 'activkey': 'myKey'}
'''

activkey = '0e11e1d65a021d0a883f309f813d74ad'
header = {"Authorization": "Bearer "+activkey}

#with open('chordProbs3new.json', "r") as f:
with open('probs4.json', "r") as f:
    data3 = json.load(f)

#data3.reverse()
#random.shuffle(data3)

histo = [0]*5000

#filename3s = 'chordSongs3.json'
#f = open(filename3s, 'w')
filename3se = 'probs4e.txt'
f2 = open(filename3se, 'w')

#f.write("[")
try:
    genre_dict # don't overwrite it if it exists for future jsons
except:
    genre_dict = {}
big_dict = {}

j = 1
totals = str(len(data3))
print(totals)

driver = webdriver.Chrome(r"C:\Users\Jesse\Desktop\work\code\PSDS4900\chromedriver\chromedriver.exe")

for d in data3:
    chord1, chord2, chord3, chord4 = d['child_path'].split(",")
    mykey = str(chord1)+','+str(chord2)+','+str(chord3)+','+str(chord4)
    print(str(j)+'/'+totals, mykey)
    song_list = []
    
    #f.write("{ \""+mykey+"\" : ")
    
    try:
        myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
        page = 1
        keepGoing = True
        #while myData.content and myData.content != b'No songs match this chord progressionnull':
        while keepGoing:
            # can't parse if it's this byte string
            if myData.content != b'No songs match this chord progressionnull':
                #print('page', page)
                # parse each song
                try:
                    my_json = myData.json()
                    songs = len(my_json)
                    #print('songs len', songs)
                    if songs:
                        # for each song, pull the genre, add it to the genre_dict and big_dict, then write it to the file
                        for i in range(songs):
                            url = my_json[i]['url'].split("#")[0]
                            # check if already in genre_dict to save us from having to load the page slowly and parse
                            if url in genre_dict:
                                my_json[i]['genre'] = genre_dict[url]
                            # if not in dict, we need to parse page and add to dict
                            else:
                                driver.get(url)
                                html = driver.page_source
                                soup = BeautifulSoup(html, 'html.parser')
                                try:
                                    #genre = soup.find_all(['multiselect'])[0].find('a').contents[0]
                                    # some songs don't have a genre and some have multiple genres...
                                    genres = soup.find_all(['multiselect'])[0].find_all('a')
                                    gk = 0
                                    glist = []
                                    for g in genres:
                                        # .find_all() will return a lot of garbage, but the genres will have a length of 1
                                        if len(g) > 1:
                                            # if even the first is garbage, it has no genre
                                            if gk == 0:
                                                my_json[i]['genre'] = ["Unknown"]
                                                genre_dict[url] = ["Unknown"]
                                            break
                                        else:
                                            glist.append(g.contents[0])
                                        gk += 1
                                    my_json[i]['genre'] = glist
                                    genre_dict[url] = glist
                                except:
                                    print('Unable to extract genre for', url)
                                    my_json[i]['genre'] = ["Unknown"]
                                    genre_dict[url] = ["Unknown"]
                                #driver.close()
                                # clean up and append song to list
                                my_json[i].pop('section', None)
                                my_json[i]['url'] = url

                                #f.write('\n'+str(my_json[i])+',')
                                song_list.append(my_json[i])

                        if songs == 20:
                            #keepGoing = True
                            page += 1
                            myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                        else:
                            # update histogram of how many songs per chord progression
                            index = page*20 - 20 + songs
                            print('histo', index)
                            histo[index] += 1
                            keepGoing = False
                    else:
                        # update histogram of how many songs per chord progression
                        index = page*20 - 20
                        print("histo", index)
                        histo[index] += 1
                        keepGoing = False

                except:
                    # for some reason, page=20 doesn't work on 1-5-6-4...
                    print('Unable to extract for', chord1, chord2, chord3, chord4, "page", page)
                    f2.write('\n\nUnable to extract for '+chord1+','+chord2+','+chord3+','+chord4)
                    f2.close()
                    f2 = open(filename3se, 'a')

                    page += 1
                    myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
            else:
                keepGoing = False

    except:
        print('Unable to extract for', chord1, chord2, chord3, chord4)
        f2.write('\n\nUnable to extract for '+chord1+','+chord2+','+chord3+','+chord4)
        f2.close()
        f2 = open(filename3se, 'a')
        
    big_dict[mykey] = song_list
    #f.close()
    #f = open(filename3s, 'a')
    #time.sleep(1.1)
    j += 1
    
f2.close()

filename3s = 'probs4genre.json'
with open(filename3s, "w") as f:  
    json.dump(big_dict, f) 

tend = time.time()
print(tend)
#f.close()
print(tend-tstart)

print(histo)
with open("histo.txt", "w") as f:  
    f.write(histo)

1614917391.0305016
5061
1/5061 6,56/6,543/4,47
histo 1
2/5061 57/6,6,M67,264
histo 1
3/5061 5/6,6,5/6,6
histo 23
4/5061 66,5/5,564/5,5
histo 1
5/5061 4,16,27,5/6
histo 1
6/5061 2,5,57,2
histo 1
7/5061 2,b7,5,6
histo 1
8/5061 1,564/6,6,47
histo 1
9/5061 27,47,5/5,57
histo 1
10/5061 57/5,b27,57,1
histo 1
11/5061 2,37,47,5
histo 1
12/5061 4,27,6,642
histo 1
13/5061 7,4,7,6
histo 1
14/5061 1,5,6,664
histo 2
15/5061 27,57,b57,57
histo 2
16/5061 b7,464,1,6
histo 1
17/5061 164,46,2,1
histo 1
18/5061 5/5,5,4,16
histo 1
19/5061 4,1,5/5,5
histo 8
20/5061 6,4,6,26
histo 1
21/5061 67,1,27,4
histo 2
22/5061 26,164,542,16
histo 1
23/5061 5/6,6,b7,6
histo 2
24/5061 464,27,464,46
histo 1
25/5061 5/5,4,1,564
histo 1
26/5061 4,5,564,6
histo 2
27/5061 4,b4,b442,Y2
histo 1
28/5061 365,27,47,6
histo 1
29/5061 37,2,57,1
histo 2
30/5061 5,6,5,5/5
histo 2
31/5061 1,5/5,5/6,6
histo 5
32/5061 16,b6,16,b6
histo 1
33/5061 3,6,47,543/3
histo 1
34/5061 b7,4,b1,D4
histo 1
35/5061 36,4,47,3
histo 1
36/5061 5,46,6,67


Unable to extract for 27 67 37 27 page 12
Unable to extract for 27 67 37 27 page 13
Unable to extract for 27 67 37 27 page 14
Unable to extract for 27 67 37 27 page 15
Unable to extract for 27 67 37 27 page 16
Unable to extract for 27 67 37 27 page 17
Unable to extract for 27 67 37 27 page 18
Unable to extract for 27 67 37 27 page 19
Unable to extract for 27 67 37 27 page 20
Unable to extract for 27 67 37 27 page 21
Unable to extract for 27 67 37 27 page 22
Unable to extract for 27 67 37 27 page 23
Unable to extract for 27 67 37 27 page 24
Unable to extract for 27 67 37 27 page 25
Unable to extract for 27 67 37 27 page 26
Unable to extract for 27 67 37 27 page 27
Unable to extract for 27 67 37 27 page 28
Unable to extract for 27 67 37 27 page 29
Unable to extract for 27 67 37 27 page 30
Unable to extract for 27 67 37 27 page 31
Unable to extract for 27 67 37 27 page 32
Unable to extract for 27 67 37 27 page 33
Unable to extract for 27 67 37 27 page 34
Unable to extract for 27 67 37 27 

Unable to extract for 27 67 37 27 page 205
Unable to extract for 27 67 37 27 page 206
Unable to extract for 27 67 37 27 page 207
Unable to extract for 27 67 37 27 page 208
Unable to extract for 27 67 37 27 page 209
Unable to extract for 27 67 37 27 page 210
Unable to extract for 27 67 37 27 page 211
Unable to extract for 27 67 37 27 page 212
Unable to extract for 27 67 37 27 page 213
Unable to extract for 27 67 37 27 page 214
Unable to extract for 27 67 37 27 page 215
Unable to extract for 27 67 37 27 page 216
Unable to extract for 27 67 37 27 page 217
Unable to extract for 27 67 37 27 page 218
Unable to extract for 27 67 37 27 page 219
Unable to extract for 27 67 37 27 page 220
Unable to extract for 27 67 37 27 page 221
Unable to extract for 27 67 37 27 page 222
Unable to extract for 27 67 37 27 page 223
Unable to extract for 27 67 37 27 page 224
Unable to extract for 27 67 37 27 page 225
Unable to extract for 27 67 37 27 page 226
Unable to extract for 27 67 37 27 page 227
Unable to e

Unable to extract for 27 67 37 27 page 396
Unable to extract for 27 67 37 27 page 397
Unable to extract for 27 67 37 27 page 398
Unable to extract for 27 67 37 27 page 399
Unable to extract for 27 67 37 27 page 400
Unable to extract for 27 67 37 27 page 401
Unable to extract for 27 67 37 27 page 402
Unable to extract for 27 67 37 27 page 403
Unable to extract for 27 67 37 27 page 404
Unable to extract for 27 67 37 27 page 405
Unable to extract for 27 67 37 27 page 406
Unable to extract for 27 67 37 27 page 407
Unable to extract for 27 67 37 27 page 408
Unable to extract for 27 67 37 27 page 409
Unable to extract for 27 67 37 27 page 410
Unable to extract for 27 67 37 27 page 411
Unable to extract for 27 67 37 27 page 412
Unable to extract for 27 67 37 27 page 413
Unable to extract for 27 67 37 27 page 414
Unable to extract for 27 67 37 27 page 415
Unable to extract for 27 67 37 27 page 416
Unable to extract for 27 67 37 27 page 417
Unable to extract for 27 67 37 27 page 418
Unable to e

Unable to extract for 27 67 37 27 page 587
Unable to extract for 27 67 37 27 page 588
Unable to extract for 27 67 37 27 page 589
Unable to extract for 27 67 37 27 page 590
Unable to extract for 27 67 37 27 page 591
Unable to extract for 27 67 37 27 page 592
Unable to extract for 27 67 37 27 page 593
Unable to extract for 27 67 37 27 page 594
Unable to extract for 27 67 37 27 page 595
Unable to extract for 27 67 37 27 page 596
Unable to extract for 27 67 37 27 page 597
Unable to extract for 27 67 37 27 page 598
Unable to extract for 27 67 37 27 page 599
Unable to extract for 27 67 37 27 page 600
Unable to extract for 27 67 37 27 page 601
Unable to extract for 27 67 37 27 page 602
Unable to extract for 27 67 37 27 page 603
Unable to extract for 27 67 37 27 page 604
Unable to extract for 27 67 37 27 page 605
Unable to extract for 27 67 37 27 page 606
Unable to extract for 27 67 37 27 page 607
Unable to extract for 27 67 37 27 page 608
Unable to extract for 27 67 37 27 page 609
Unable to e

...

Unable to extract for 27 67 37 27 page 2091
Unable to extract for 27 67 37 27 page 2092
Unable to extract for 27 67 37 27 page 2093
Unable to extract for 27 67 37 27 page 2094
Unable to extract for 27 67 37 27 page 2095
Unable to extract for 27 67 37 27 page 2096
Unable to extract for 27 67 37 27 page 2097
Unable to extract for 27 67 37 27 page 2098
Unable to extract for 27 67 37 27 page 2099
Unable to extract for 27 67 37 27 page 2100
Unable to extract for 27 67 37 27 page 2101
Unable to extract for 27 67 37 27 page 2102
Unable to extract for 27 67 37 27 page 2103
Unable to extract for 27 67 37 27 page 2104
Unable to extract for 27 67 37 27 page 2105
Unable to extract for 27 67 37 27 page 2106
Unable to extract for 27 67 37 27 page 2107
Unable to extract for 27 67 37 27 page 2108
Unable to extract for 27 67 37 27 page 2109
Unable to extract for 27 67 37 27 page 2110
Unable to extract for 27 67 37 27 page 2111
Unable to extract for 27 67 37 27 page 2112
Unable to extract for 27 67 37 2

Unable to extract for 27 67 37 27 page 2278
Unable to extract for 27 67 37 27 page 2279
Unable to extract for 27 67 37 27 page 2280
Unable to extract for 27 67 37 27 page 2281
Unable to extract for 27 67 37 27 page 2282
Unable to extract for 27 67 37 27 page 2283
Unable to extract for 27 67 37 27 page 2284
Unable to extract for 27 67 37 27 page 2285
Unable to extract for 27 67 37 27 page 2286
Unable to extract for 27 67 37 27 page 2287
Unable to extract for 27 67 37 27 page 2288
Unable to extract for 27 67 37 27 page 2289
Unable to extract for 27 67 37 27 page 2290
Unable to extract for 27 67 37 27 page 2291
Unable to extract for 27 67 37 27 page 2292
Unable to extract for 27 67 37 27 page 2293
Unable to extract for 27 67 37 27 page 2294
Unable to extract for 27 67 37 27 page 2295
Unable to extract for 27 67 37 27 page 2296
Unable to extract for 27 67 37 27 page 2297
Unable to extract for 27 67 37 27 page 2298
Unable to extract for 27 67 37 27 page 2299
Unable to extract for 27 67 37 2

Unable to extract for 27 67 37 27 page 2465
Unable to extract for 27 67 37 27 page 2466
Unable to extract for 27 67 37 27 page 2467
Unable to extract for 27 67 37 27 page 2468
Unable to extract for 27 67 37 27 page 2469
Unable to extract for 27 67 37 27 page 2470
Unable to extract for 27 67 37 27 page 2471
Unable to extract for 27 67 37 27 page 2472
Unable to extract for 27 67 37 27 page 2473
Unable to extract for 27 67 37 27 page 2474
Unable to extract for 27 67 37 27 page 2475
Unable to extract for 27 67 37 27 page 2476
Unable to extract for 27 67 37 27 page 2477
Unable to extract for 27 67 37 27 page 2478
Unable to extract for 27 67 37 27 page 2479
Unable to extract for 27 67 37 27 page 2480
Unable to extract for 27 67 37 27 page 2481
Unable to extract for 27 67 37 27 page 2482
Unable to extract for 27 67 37 27 page 2483
Unable to extract for 27 67 37 27 page 2484
Unable to extract for 27 67 37 27 page 2485
Unable to extract for 27 67 37 27 page 2486
Unable to extract for 27 67 37 2

Unable to extract for 27 67 37 27 page 2652
Unable to extract for 27 67 37 27 page 2653
Unable to extract for 27 67 37 27 page 2654
Unable to extract for 27 67 37 27 page 2655
Unable to extract for 27 67 37 27 page 2656
Unable to extract for 27 67 37 27 page 2657
Unable to extract for 27 67 37 27 page 2658
Unable to extract for 27 67 37 27 page 2659
Unable to extract for 27 67 37 27 page 2660
Unable to extract for 27 67 37 27 page 2661
Unable to extract for 27 67 37 27 page 2662
Unable to extract for 27 67 37 27 page 2663
Unable to extract for 27 67 37 27 page 2664
Unable to extract for 27 67 37 27 page 2665
Unable to extract for 27 67 37 27 page 2666
Unable to extract for 27 67 37 27 page 2667
Unable to extract for 27 67 37 27 page 2668
Unable to extract for 27 67 37 27 page 2669
Unable to extract for 27 67 37 27 page 2670
Unable to extract for 27 67 37 27 page 2671
Unable to extract for 27 67 37 27 page 2672
Unable to extract for 27 67 37 27 page 2673
Unable to extract for 27 67 37 2

Unable to extract for 27 67 37 27 page 2839
Unable to extract for 27 67 37 27 page 2840
Unable to extract for 27 67 37 27 page 2841
Unable to extract for 27 67 37 27 page 2842
Unable to extract for 27 67 37 27 page 2843
Unable to extract for 27 67 37 27 page 2844
Unable to extract for 27 67 37 27 page 2845
Unable to extract for 27 67 37 27 page 2846
Unable to extract for 27 67 37 27 page 2847
Unable to extract for 27 67 37 27 page 2848
Unable to extract for 27 67 37 27 page 2849
Unable to extract for 27 67 37 27 page 2850
Unable to extract for 27 67 37 27 page 2851
Unable to extract for 27 67 37 27 page 2852
Unable to extract for 27 67 37 27 page 2853
Unable to extract for 27 67 37 27 page 2854
Unable to extract for 27 67 37 27 page 2855
Unable to extract for 27 67 37 27 page 2856
Unable to extract for 27 67 37 27 page 2857
Unable to extract for 27 67 37 27 page 2858
Unable to extract for 27 67 37 27 page 2859
Unable to extract for 27 67 37 27 page 2860
Unable to extract for 27 67 37 2